In [12]:
%pip install pandas
%pip install lightgbm
%pip install scikit-learn
%pip install optuna
import glob
import pandas as pd
import numpy as np
# import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae
import optuna.integration.lightgbm as lgb_tune
import optuna.integration.lightgbm as lgb


files = glob.glob("train/*.csv")
data_list = []
for file in files:
    data_list.append(pd.read_csv(file, index_col=0))
df = pd.concat(data_list)
# df.isnull().sum()
df.head()
# df.describe()
# df.groupby("都道府県名").count()["市区町村名"].sort_values(ascending=False)
# df.pivot_table(index="都道府県名", columns="市区町村名", values="取引価格（総額）_log", aggfunc="count").fillna(0).astype(int)

/tmp/ipykernel_79381/1961842341.py:17: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0))
/tmp/ipykernel_79381/1961842341.py:17: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0))


,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,...,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
ID,,,,,,,,,,,,,,,,,,,,,
13170256,中古マンション等,NaN,13120,東京都,練馬区,錦,上板橋,12,１Ｋ,15,...,NaN,NaN,NaN,近隣商業地域,80.0,300.0,2019年第４四半期,未改装,NaN,6.812913
13106958,中古マンション等,NaN,13108,東京都,江東区,新砂,南砂町,4,２ＬＤＫ,65,...,NaN,NaN,NaN,第１種中高層住居専用地域,60.0,300.0,2014年第３四半期,未改装,NaN,7.518514
13098062,中古マンション等,NaN,13102,東京都,中央区,日本橋浜町,浜町,2,２ＬＤＫ,55,...,NaN,NaN,NaN,商業地域,80.0,700.0,2006年第２四半期,未改装,NaN,7.633468
13243242,中古マンション等,NaN,13204,東京都,三鷹市,下連雀,三鷹,10,１Ｋ,15,...,NaN,NaN,NaN,第１種住居地域,60.0,300.0,2018年第３四半期,NaN,NaN,6.892095
13302520,中古マンション等,NaN,13120,東京都,練馬区,上石神井,上石神井,4,オープンフロア,45,...,NaN,NaN,NaN,近隣商業地域,80.0,300.0,2012年第３四半期,未改装,NaN,7.556303


In [13]:
import glob
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae

files = glob.glob("train/*.csv")
data_list = []
for file in files:
    data_list.append(pd.read_csv(file, index_col=0))
df = pd.concat(data_list)
# df.isnull().sum()
df.head()
# df.describe()
# df.groupby("都道府県名").count()["市区町村名"].sort_values(ascending=False)
# df.pivot_table(index="都道府県名", columns="間取り", values="面積（㎡）", aggfunc="count").fillna(0).astype(int)

/tmp/ipykernel_79381/4004485803.py:11: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0))
/tmp/ipykernel_79381/4004485803.py:11: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0))


,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,...,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
ID,,,,,,,,,,,,,,,,,,,,,
13170256,中古マンション等,NaN,13120,東京都,練馬区,錦,上板橋,12,１Ｋ,15,...,NaN,NaN,NaN,近隣商業地域,80.0,300.0,2019年第４四半期,未改装,NaN,6.812913
13106958,中古マンション等,NaN,13108,東京都,江東区,新砂,南砂町,4,２ＬＤＫ,65,...,NaN,NaN,NaN,第１種中高層住居専用地域,60.0,300.0,2014年第３四半期,未改装,NaN,7.518514
13098062,中古マンション等,NaN,13102,東京都,中央区,日本橋浜町,浜町,2,２ＬＤＫ,55,...,NaN,NaN,NaN,商業地域,80.0,700.0,2006年第２四半期,未改装,NaN,7.633468
13243242,中古マンション等,NaN,13204,東京都,三鷹市,下連雀,三鷹,10,１Ｋ,15,...,NaN,NaN,NaN,第１種住居地域,60.0,300.0,2018年第３四半期,NaN,NaN,6.892095
13302520,中古マンション等,NaN,13120,東京都,練馬区,上石神井,上石神井,4,オープンフロア,45,...,NaN,NaN,NaN,近隣商業地域,80.0,300.0,2012年第３四半期,未改装,NaN,7.556303


In [14]:
def data_pre(df):
    nonnull_list = []
    for col in df.columns:
        nonnull = df[col].count()
        if nonnull == 0:
            nonnull_list.append(col)
    # print("データが全てnull",nonnull_list)

    df = df.drop(nonnull_list, axis=1)

    df = df.drop("市区町村名", axis=1)

    df = df.drop("種類", axis=1)

    dis = {
        "30分?60分":45,
        "1H?1H30":75,
        "2H?":120,
        "1H30?2H":105
    }
    df["最寄駅：距離（分）"] = df["最寄駅：距離（分）"].replace(dis).astype(float)

    df["面積（㎡）"] = df["面積（㎡）"].replace("2000㎡以上", 2000).astype(float)


    y_list = {}
    for i in df["建築年"].value_counts().keys():
        if "平成" in i:
            num = float(i.split("平成")[1].split("年")[0])
            year = 33 - num
        if "令和" in i:
            num = float(i.split("令和")[1].split("年")[0])
            year = 3 - num
        if "昭和" in i:
            num = float(i.split("昭和")[1].split("年")[0])
            year = 96 - num
        y_list[i] = year
    y_list["戦前"] = 76
    df["建築年"] = df["建築年"].replace(y_list)

    year = {
        "年第１四半期": ".25",
        "年第２四半期": ".50",
        "年第３四半期": ".75",
        "年第４四半期": ".99"
    }
    year_list = {}
    for i in df["取引時点"].value_counts().keys():
        for k, j in year.items():
            if k in i:
                year_rep = i.replace(k, j)
        year_list[i] = year_rep
    df["取引時点"] = df["取引時点"].replace(year_list).astype(float)

    for col in ["都道府県名", "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"]:
                df[col] = df[col].astype("category")

    return df

df = data_pre(df)
df.head()

,市区町村コード,都道府県名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
ID,,,,,,,,,,,,,,,,,,
13170256,13120,東京都,錦,上板橋,12.0,１Ｋ,15.0,33.0,ＲＣ,NaN,住宅,近隣商業地域,80.0,300.0,2019.99,未改装,NaN,6.812913
13106958,13108,東京都,新砂,南砂町,4.0,２ＬＤＫ,65.0,16.0,ＲＣ,住宅,住宅,第１種中高層住居専用地域,60.0,300.0,2014.75,未改装,NaN,7.518514
13098062,13102,東京都,日本橋浜町,浜町,2.0,２ＬＤＫ,55.0,16.0,ＲＣ,住宅,NaN,商業地域,80.0,700.0,2006.50,未改装,NaN,7.633468
13243242,13204,東京都,下連雀,三鷹,10.0,１Ｋ,15.0,27.0,ＲＣ,NaN,住宅,第１種住居地域,60.0,300.0,2018.75,NaN,NaN,6.892095
13302520,13120,東京都,上石神井,上石神井,4.0,オープンフロア,45.0,9.0,ＲＣ,店舗,NaN,近隣商業地域,80.0,300.0,2012.75,未改装,NaN,7.556303


In [20]:
df_train, df_val =train_test_split(df, test_size=0.2)

col = "取引価格（総額）_log"
train_y = df_train[col]
train_x = df_train.drop(col, axis=1)

val_y = df_val[col]
val_x = df_val.drop(col, axis=1)

trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)


params = {
    "objective": "regression",
    "metrics": "mae"
}
# model_tune = lgb_tune.train(
#     params, trains, valid_sets=valids, 
#     verbose_eval=100, 
#     early_stopping_rounds=100,
# )
verbose_eval = 0

# model = lgb.train(model_tune.params, trains, valid_sets=valids, num_boost_round=1000, early_stopping_rounds=100)
model = lgb.train(params, trains, valid_sets=valids, num_boost_round=1000, callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=True),lgb.log_evaluation(verbose_eval)])

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.060267 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11664
[LightGBM] [Info] Number of data points in the train set: 509880, number of used features: 17
[LightGBM] [Info] Start training from score 7.217036
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[998]	valid_0's l1: 0.0764961


In [ ]:
df_test = pd.read_csv("test.csv", index_col=0)
df_test = data_pre(df_test)

predict = model.predict(df_test)
df_test["取引価格（総額）_log"] = predict
df_test[["取引価格（総額）_log"]].to_csv("submit_test.csv")